In [11]:
# import
import importlib
import pandas as pd
from pipeline_executor import PipelineExecutor
import nlp_analysis.word_wizard as ww

# Supress warnings
import warnings
warnings.filterwarnings('ignore')

In [12]:
importlib.reload(ww)

<module 'nlp_analysis.word_wizard' from '/Users/johnbergmann/Developer/MIBA-2023-CAPSTONE-RB-NLP/nlp_analysis/word_wizard.py'>

In [13]:
# get data
topic = "Quantum Computing Research"
pipeline_executor = PipelineExecutor()
edge = pipeline_executor.execute(query=topic, max_articles=None, overwrite=False)

In [14]:
# Paragraphs - Word - silhouette
wizard = ww.WordWizard(edge, interest="body")

In [15]:
wizard.create_sentence_embeddings() \
.cluster_embeddings() \
.entitiy_recognition() \
.summarize_medoids() \
.find_sentiment(device="cpu") \
.topic_modelling()

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

Extracting organizations from body:   0%|          | 0/5 [00:00<?, ?it/s]

Creating summaries for cluster medoids based on body:   0%|          | 0/10 [00:00<?, ?it/s]

Calculating sentiment using body:   0%|          | 0/371 [00:00<?, ?it/s]

In [16]:
wizard.reduce_demensionality()

In [23]:
wizard.df.head()

,engine,link,source,title,description,body,body_sentence_embeddings,body_sentence_embeddings_clusters,body_sentence_embeddings_clusters_medoids,body_clusters_sentence_embeddings_NER,body_sentence_embeddings_clusters_medoids_summaries,body_sentiment,topics,body_reduced_dimensions_word_embeddings
0,Yahoo,https://www.etfdailynews.com/2023/06/21/quantu...,ETF DAILY NEWS,Quantum Computing (OTCMKTS:QUBT) Shares Down 8.8%,"Quantum Computing, Inc. (OTCMKTS:QUBT – Get Ra...","Quantum Computing, Inc. (OTCMKTS:QUBT – Get Ra...","[-0.008077695965766907, -0.039184171706438065,...",3,False,"['IBM', 'Photonic Integrated Circuit', 'Electr...",NaN,0.0,"[(ic, 0.0012261072687060005), (ic quantum, 0.0...","[4.445281505584717, 17.517343521118164]"
1,Yahoo,https://www.digitaljournal.com/pr/news/getnews...,Digital Journal,2023 Quantum Computing in Healthcare Global Ma...,"""IBM (US), Google, Inc. (US), Rigetti & Co, LL...","PRESS RELEASE Published June 22, 2023 GetNews ...","[-0.06863654404878616, -0.04752199351787567, -...",3,False,"['IBM', 'Photonic Integrated Circuit', 'Electr...",NaN,2.0,"[(ic, 0.0012261072687060005), (ic quantum, 0.0...","[4.433454990386963, 17.301395416259766]"
2,Yahoo,https://www.nbc11news.com/prnewswire/2023/06/2...,KKCO Grand Junction,Infleqtion Wins Funding to Develop and Manufac...,"Infleqtion, the world's quantum information co...","OXFORD, England, June 26, 2023 /PRNewswire/ --...","[-0.061800699681043625, -0.061217259615659714,...",4,False,"['IBM', 'NVIDIA', 'Cleveland Clinic', 'Microso...",NaN,2.0,"[(cleveland, 0.001042235672849059), (clinic, 0...","[3.327619791030884, 15.931803703308105]"
3,Bing,https://www.benzinga.com/pressreleases/23/06/n...,Benzinga.com,Quantum Computing Market size to grow by USD 9...,The quantum computing market size is expected ...,"NEW YORK, June 20, 2023 /PRNewswire/ -- The qu...","[0.018597137182950974, -0.053545031696558, -0....",3,False,"['IBM', 'Photonic Integrated Circuit', 'Electr...",NaN,2.0,"[(ic, 0.0012261072687060005), (ic quantum, 0.0...","[5.001786231994629, 17.530866622924805]"
4,Yahoo,https://finance.yahoo.com/news/quantum-compute...,The Independent via Yahoo Finance,Quantum computers to overtake regular computer...,Quantum computers have the potential to be ord...,A qubit test chip (blue) on a Microsoft quantu...,"[-0.08655066043138504, -0.0048426971770823, -0...",1,True,"['IBM', 'Intel', 'Microsoft', 'Google', 'NVIDIA']",Quantum computers have the potential to be ord...,2.0,"[(tunnel fall, 0.0011373903645516944), (tunnel...","[0.9095239639282227, 16.933956146240234]"


In [ ]:
# get unique values from wizard.df[['topics']]
wizard.df[['topics']].unique()

In [ ]:
wizard.df['x'] = wizard.df.iloc[:,-1].apply(lambda x: x[0])
wizard.df['y'] = wizard.df.iloc[:,-2].apply(lambda x: x[1])

# count cluster size
wizard.df['cluster_size'] = wizard.df.groupby('paragraph_sentence_embeddings_clusters')['paragraph_sentence_embeddings_clusters'].transform('count')

# aggregate cluster sentiment
wizard.df['cluster_sentiment'] = wizard.df.groupby('paragraph_sentence_embeddings_clusters')['paragraph_sentiment'].transform('mean')

# keep only medoids
wizard.df = wizard.df[wizard.df['paragraph_sentence_embeddings_clusters_medoids'] == True]

# keep only cluster_size, x, y, cluster_sentiment, paragraph_clusters_sentence_embeddings_NER, topics
ner_col = [col for col in wizard.df.columns if col.endswith('NER')][0]
wizard.df = wizard.df[['cluster_size', 'x', 'y', 'cluster_sentiment', ner_col, 'topics']]

# rename columns
wizard.df.columns = ['size', 'x', 'y', 'sentiment', 'entities', 'topics']

In [ ]:
wizard.df.columns = ['size', 'x', 'y', 'sentiment', 'entities', 'topics']
wizard.df.to_csv('data/RB.csv', index=False)

In [ ]:
# Paragraphs - Sentence - silhouette
wizard2 = ww.WordWizard(df=quantum, interest = 'paragraph')
wizard2.create_word_embeddings() \
    .cluster_embeddings() \
    .entitiy_recognition() \
    .summarize_medoids() \
    .find_sentiment(lean=False, device="cpu") \
    .topic_modelling()

In [ ]:
# Article - Word - silhouette
wizard3 = ww.WordWizard(df=quantum, interest = 'body')
wizard3.create_word_embeddings() \
    .cluster_embeddings() \
    .entitiy_recognition() \
    .summarize_medoids() \
    .find_sentiment(lean=False, device="cpu") \
    .topic_modelling()

In [ ]:
# Article - Sentence - silhouette
wizard4 = ww.WordWizard(df=quantum, interest = 'body')
wizard4.create_sentence_embeddings() \
    .cluster_embeddings() \
    .entitiy_recognition() \
    .summarize_medoids() \
    .find_sentiment(lean=False, device="cpu") \
    .topic_modelling()

In [ ]:
wizard.df.to_csv("data/para_word_sil.csv", index=False)
wizard2.df.to_csv("data/para_sent_sil.csv", index=False)
wizard3.df.to_csv("data/art_word_sil.csv", index=False)
wizard4.df.to_csv("data/art_sent_sil.csv", index=False)

# Gaining Insights about Quantum Computing from News NLP

In [ ]:
para_word_sil = pd.read_csv("data/para_word_sil.csv")
para_sent_sil = pd.read_csv("data/para_sent_sil.csv")
art_word_sil = pd.read_csv("data/art_word_sil.csv")
art_sent_sil = pd.read_csv("data/art_sent_sil.csv")

In [ ]:
import ast
def summarize(df, interest):
    wizard = ww.WordWizard(df=df, interest = interest)

    if 'paragraph_sentence_embeddings' in wizard.df.columns:
        embed = 'paragraph_sentence_embeddings'
    elif 'paragraph_word_embeddings' in wizard.df.columns:
        embed = 'paragraph_word_embeddings'
    elif 'body_sentence_embeddings' in wizard.df.columns:
        embed = 'body_sentence_embeddings'
    elif 'body_word_embeddings' in wizard.df.columns:
        embed = 'body_word_embeddings'

    # str to list
    try:
        wizard.df[embed] = wizard.df[embed].apply(ast.literal_eval)
        
        # reduce demensionality
        wizard.reduce_demensionality()

        # put x and y in different columns
        wizard.df['x'] = art_sent_sil.iloc[:,-1].apply(lambda x: x[0])
        wizard.df['y'] = art_sent_sil.iloc[:,-1].apply(lambda x: x[1])
        
        # count cluster size
        wizard.df['cluster_size'] = wizard.df.groupby(embed + '_clusters')[embed + '_clusters'].transform('count')
        
        # aggregate cluster sentiment
        wizard.df['cluster_sentiment'] = wizard.df.groupby(embed + '_clusters')['paragraph_sentiment'].transform('mean')
        
        # keep only medoids
        wizard.df = wizard.df[wizard.df[embed + '_clusters' + '_medoids'] == True]
        
        # keep only cluster_size, x, y, cluster_sentiment, paragraph_clusters_sentence_embeddings_NER, topics
        ner_col = [col for col in wizard.df.columns if col.endswith('NER')][0]
        wizard.df = wizard.df[['cluster_size', 'x', 'y', 'cluster_sentiment', ner_col, 'topics']]
        
        # rename columns
        wizard.df.columns = ['size', 'x', 'y', 'sentiment', 'entities', 'topics']

    except:
        # count cluster size
        wizard.df['cluster_size'] = wizard.df.groupby(embed + '_clusters')[embed + '_clusters'].transform('count')
        
        # aggregate cluster sentiment
        wizard.df['cluster_sentiment'] = wizard.df.groupby(embed + '_clusters')['paragraph_sentiment'].transform('mean')
        
        # keep only medoids
        wizard.df = wizard.df[wizard.df[embed + '_clusters' + '_medoids'] == True]
        
        # keep only cluster_size, cluster_sentiment, paragraph_clusters_sentence_embeddings_NER, topics
        ner_col = [col for col in wizard.df.columns if col.endswith('NER')][0]
        wizard.df = wizard.df[['cluster_size', 'cluster_sentiment', ner_col, 'topics']]
        
        # rename columns
        wizard.df.columns = ['size', 'sentiment', 'entities', 'topics']

    
    return wizard.df

In [ ]:
for df in para_word_sil, para_sent_sil:
    df = summarize(df, interest = 'paragraph')

for df in art_word_sil, art_sent_sil:
    df = summarize(df, interest = 'body')